In [1]:
### PFX prototyping

In [2]:
import pandas as pd
import os
from readability import Readability
import textstat
from openai import OpenAI
import json
import re
from pfx_fewshots import baseline_fewshot_prompt,example,baseline_fewshot_icd10_labeling_prompt,icd10_example,baseline_zeroshot_prompt
client = OpenAI()

In [3]:
df_fewshot = pd.read_csv('pfx_fewshot_examples_college.csv')

In [4]:
df_fewshot.head()


,Body_Part,Incidental_Finding,ICD10_code,PFx,PFx_ICD10_code
0,Head,Cerebral cavernous malformation,I67.1,A small cavernous malformation of the brain is...,I67.1
1,Head,Infundibulum of cerebral artery,Q28.2,An infundibulum detected on an MRI of the brai...,Q28.2
2,Neck,Tornwaldt cyst,J39.2,"A Tornwaldt cyst, often detected incidentally ...",J39.2
3,Head,Fluid in the right petrous apex,H70.8,A small amount of incidentally found fluid in ...,H70.8
4,Abdomen,Spinal hemangioma,D18.09,"A spinal hemangioma, often found incidentally ...",D18.09


In [5]:
df_eval = pd.read_csv('pfx_evaluation_data.csv')

In [6]:
df_eval.head()

,Body_Part,Incidental_Finding,ICD10_code,Unnamed: 3
0,Head,White matter lesions,R90.82,NaN
1,Head,Arachnoid cyst,Q04.3,NaN
2,Head,Pituitary microadenoma,D35.2,NaN
3,Head,Pineal cyst,Q04.6,NaN
4,Head,Chiari I malformation,Q07.0,NaN


In [7]:
# https://en.wikipedia.org/wiki/Flesch%E2%80%93Kincaid_readability_tests#Flesch_reading_ease
def map_reading_level(flesch_reading_ease):
    if flesch_reading_ease < 10:
        return "Professional"
    elif 10.0 <= flesch_reading_ease < 30.0:
        return "College graduate"
    elif 30.0 <= flesch_reading_ease < 50.0:
        return "College"
    elif 50.0 <= flesch_reading_ease < 60.0:
        return "10th to 12th grade"
    elif 60.0 <= flesch_reading_ease < 70.0:
        return "8th & 9th grade"
    elif 70.0 <= flesch_reading_ease < 80.0:
        return "7th grade"
    elif 80.0 <= flesch_reading_ease < 90.0:
        return "6th grade"
    elif 90.0 <= flesch_reading_ease < 100.0:
        return "5th grade"
    else:
        return ""

In [8]:
def extract_json(openai_response):
    json_match = re.search(r'```.*(\{.*?\}).*```', openai_response.message.content, re.DOTALL)
    if json_match:
        json_str = json_match.group(1)
        # Convert the extracted string to a JSON object
        json_object = json.loads(json_str.replace('\n',''))
        #print(json_object)
        return json_object
    else:
        print("No JSON object found.") 
        return dict()

In [9]:
def label_icd10s(pfx_outputs_json):
    pfx_icd10_fewshot_examples = ""
    for i,row in df_fewshot.iterrows():
        pfx_icd10_fewshot_examples += icd10_example.format(**row)
    
    pfx_icd10_codes = []
    for pfx_output in pfx_outputs_json:
        try:
            prompt = baseline_fewshot_icd10_labeling_prompt.format(examples=pfx_icd10_fewshot_examples,PFx=pfx_output['PFx'])
        except:
            print(pfx_output)
        pfx_icd10_response = client.chat.completions.create(
                #model="gpt-4o-mini",
                model="gpt-4o",
                temperature=0.0,
                messages=[
                    {"role": "system", "content": "You are an ICD10 medical coder for incidental findings." },
                    {"role": "user", "content": prompt }
                ],
                stream=False,
            )
        pfx_icd10_codes.append(pfx_icd10_response.choices[0])
    return list(map(extract_json,pfx_icd10_codes))

In [10]:
pfx_zeroshot_outputs = []
for i,row in df_eval.iloc[:].iterrows():
    #print(baseline_fewshot_prompt.format(examples=pfx_fewshot_examples,Incidental_Finding=row['Incidental_Finding']))
    prompt = baseline_zeroshot_prompt.format(Incidental_Finding=row['Incidental_Finding'])
    pfx_response = client.chat.completions.create(
        #model="gpt-4o-mini",
        model="gpt-4o",
        temperature=0.0,
        messages=[
            {"role": "system", "content": "You are a medical doctor rephrasing and explaining medical terminology to a patient." },
            {"role": "user", "content": prompt }
        ],
        stream=False,
    )
    
    pfx_zeroshot_outputs.append(pfx_response.choices[0])

In [24]:
pfx_zeroshot_outputs_json = list(map(extract_json,pfx_zeroshot_outputs))
#pfx_zeroshot_outputs_json


In [12]:
pfx_zeroshot_output_df = pd.DataFrame(pfx_zeroshot_outputs_json)
pfx_zeroshot_outputs_icd10_labels = label_icd10s(pfx_zeroshot_outputs_json)

In [13]:
pfx_zeroshot_output_df['agent_icd10_codes'] = list(map(lambda x: list(x.values())[0] if x else "", pfx_zeroshot_outputs_icd10_labels))

In [30]:
pfx_zeroshot_output_df["0_icd10_matches"]= pfx_zeroshot_output_df.ICD10_code == pfx_zeroshot_output_df.agent_icd10_codes
pfx_zeroshot_output_df["0_flesch"] = pfx_zeroshot_output_df['PFx'].apply(textstat.flesch_reading_ease)
pfx_zeroshot_output_df["0_reading_level"] = pfx_zeroshot_output_df['0_flesch'].apply(map_reading_level)

In [32]:
pfx_zeroshot_output_df.head()


,finding,ICD10_code,PFx,PFx_ICD10_code,agent_icd10_codes,0_icd10_matches,0_flesch,0_reading_level
0,White matter lesions,R90.82,White matter lesions are small areas of damage...,R90.82,R90.82,True,69.31,8th & 9th grade
1,Arachnoid cyst,G93.0,An arachnoid cyst is a fluid-filled sac that f...,G93.0,Q04.6,False,61.36,8th & 9th grade
2,Pituitary microadenoma,D35.2,"A pituitary microadenoma is a small, non-cance...",D35.2,D35.2,True,50.36,10th to 12th grade
3,Pineal cyst,Q04.6,A pineal cyst is a fluid-filled sac located in...,Q04.6,Q04.8,False,69.62,8th & 9th grade
4,Chiari I malformation,Q07.0,Chiari I malformation is a condition where the...,Q07.0,Q07.0,True,66.23,8th & 9th grade


In [31]:
#icd10 accuracy -- at least according to GPT4o
sum(pfx_zeroshot_output_df["0_icd10_matches"])/len(pfx_fewshot_output_df.index)

0.8

In [16]:
pfx_fewshot_examples = ""
for i,row in df_fewshot.iterrows():
    pfx_fewshot_examples += example.format(**row)

    
pfx_fewshot_outputs = []
for i,row in df_eval.iloc[:].iterrows():
    #print(baseline_fewshot_prompt.format(examples=pfx_fewshot_examples,Incidental_Finding=row['Incidental_Finding']))
    prompt = baseline_fewshot_prompt.format(examples=pfx_fewshot_examples,Incidental_Finding=row['Incidental_Finding'])
    pfx_response = client.chat.completions.create(
        #model="gpt-4o-mini",
        model="gpt-4o",
        temperature=0.0,
        messages=[
            {"role": "system", "content": "You are a medical doctor rephrasing and explaining medical terminology to a patient." },
            {"role": "user", "content": prompt }
        ],
        stream=False,
    )
    
    pfx_fewshot_outputs.append(pfx_response.choices[0])

In [18]:
pfx_fewshot_outputs_json = list(map(extract_json,pfx_fewshot_outputs))
pfx_fewshot_outputs_icd10_labels = label_icd10s(pfx_fewshot_outputs_json)


In [21]:
pfx_fewshot_output_df = pd.DataFrame(pfx_fewshot_outputs_json)
pfx_fewshot_output_df['agent_icd10_codes'] = list(map(lambda x: list(x.values())[0] if x else "",pfx_fewshot_outputs_icd10_labels))
pfx_fewshot_output_df["0_icd10_matches"]= pfx_fewshot_output_df.ICD10_code == pfx_zeroshot_output_df.agent_icd10_codes
pfx_fewshot_output_df["0_flesch"] = pfx_zeroshot_output_df['PFx'].apply(textstat.flesch_reading_ease)
pfx_fewshot_output_df["0_reading_level"] = pfx_zeroshot_output_df['0_flesch'].apply(map_reading_level)

In [22]:
list(map(lambda x: x[0]==x[1],zip(pfx_fewshot_output_df['ICD10_code'],pfx_fewshot_output_df['agent_icd10_codes'])))

[True,
 False,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 True]

In [23]:
#icd10 accuracy -- at least according to GPT4o
sum(_)/len(_)

0.8

In [52]:
pfx_fewshot_output_df['PFx'].apply(textstat.flesch_reading_ease)

0     42.92
1     55.84
2     38.66
3     42.72
4     30.40
5     53.81
6     45.56
7     48.13
8     54.22
9     59.84
10    65.56
11    38.66
12    45.56
13    47.62
14    47.12
15    50.87
16    48.33
17    42.72
18    52.19
19    52.39
20    48.30
21    45.56
22    40.69
23    41.40
24    33.75
Name: PFx, dtype: float64

In [26]:
textstat.flesch_reading_ease(output_df.iloc[0]['PFx'])

38.15